In [ ]:
import itertools
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import pandas as pd
import config
import os
import json

In [ ]:
STAGE = 'ss'
MOD_PREFIX = "mod_smallimg3"
NEPOCH = 'latest'


DATAPATH = config.OUTPUT_PATH
base_dir = config.RAW_DATA_PATH
img_path = config.SCHULTHESS_DATAPATH
proc_dir = config.PROC_DATA_PATH

# #for rawq:
feature = 'rawq'
methods = 'comb_modalities'
folder = "2026-01-17_hdbscan"
run = "run28"  

# folder = "2025-11-19_hdbscan"
# run = "run10"  
# mapping_list = [0, 4, 2, 3, -1, 1]

# feature = 'img_features'
# #for img features:
# folder = "2025-09-12_hdbscan"
# folder_date = folder.split('_')[0]
# run = "run92"

# feature = 'img_raw'
# folder = "2025-09-13_hdbscan_img"
# run = "run32"

# feature = 'agg'
# folder = "2025-08-11_hdbscan"
# run = 'run150'

anomalyscore_metric = "centre_mean"
cluster_col = "cluster_label"

folder_date = folder.split('_')[0]



if feature == 'rawq':
    filepath = os.path.join(proc_dir, folder, methods, run)
    hdbscan_df = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled.csv'))
    if methods == 'comb_modalities':
        hdbscan_df_test = pd.read_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_test_results.csv'))
elif feature == 'img_features' or feature == 'img_raw':
        filepath = os.path.join(proc_dir, "radiographic_features", folder, run)
        hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
elif feature == 'agg':
      filepath = os.path.join(proc_dir, folder, run)
      hdbscan_df = pd.read_csv(os.path.join(filepath, f'{folder}_{run}_umap_hdbscan_scaled.csv'))
        
kl = pd.read_csv(os.path.join(base_dir,  "brul_knee_annotations.csv"))
kl2 = pd.read_csv(os.path.join(base_dir, "rosand1_knee_annotations.csv"))
mri = pd.read_csv(os.path.join(base_dir, '2025-09-25_mrismall.csv'))



In [ ]:
print(hdbscan_df.shape)
print(hdbscan_df_test.shape)

In [ ]:
folder_df = "2025-09-11_data_exploration"
df_filename = "inmodi_data_questionnaire_kl_woSC.csv"
df = pd.read_csv(os.path.join(proc_dir, folder_df, df_filename))
df2 = hdbscan_df[hdbscan_df['id'].isin(df['name'])]
print(len(df2))
try:
    df2_test = hdbscan_df_test[hdbscan_df_test['id'].isin(df['name'])]
    print(len(df2_test))
except:
    pass

In [ ]:
hdbscan_df = df2
hdbscan_df_test = df2_test

In [ ]:
# l = ['IM1512_2_left',
#  'IM1512_2_right',
#  'IM1567_2_left',
#  'IM1567_2_right',
#  'IM1578_2_left',
#  'IM1578_2_right',
#  'IM2511_1_left',
#  'IM2511_1_right',
#  'IM2569_1_left',
#  'IM2569_1_right',
#  'IM3013_1_right',
#  'IM3019_1_right',
#  'IM3020_1_left',
#  'IM3020_1_right']

In [ ]:
ques = pd.read_csv(os.path.join(base_dir, 'questionnaires_raw.csv')) #?

In [ ]:
# for i in l:
#     hdbscan_df[hdbscan_df['id'] == i]

In [ ]:
hdbscan_df = hdbscan_df.merge(kl, left_on = 'id', right_on='name', how='left', validate='one_to_one')

try:
    hdbscan_df_test = hdbscan_df_test.merge(kl, left_on = 'id', right_on='name', how='left', validate='one_to_one')
except:
    pass
# hdbscan_df = hdbscan_df.merge(kl2, left_on = 'id', right_on='name', how='left', validate='one_to_one', suffixes=('', '2'))

try:
    hdbscan_df.drop(columns=['Unnamed: 0'], inplace=True)
    hdbscan_df_test.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass

df = pd.read_csv(os.path.join(DATAPATH, 'outputs', 'dfs', 'ss', 'mod_smallimg3_ss_aggregated_scores.csv'))

df['id_temp'] = df['id'].apply(lambda x: x.split('/')[-1])
df['id'] = df['id_temp'].apply(lambda x: x.split('.')[0])
df.drop(columns=['id_temp'], inplace=True)
hdbscan_df = hdbscan_df.merge(df, on='id', how='left', validate='one_to_one')

try:
    hdbscan_df_test = hdbscan_df_test.merge(df, on='id', how='left', validate='one_to_one')
    hdbscan_df_test['KL-Score'].fillna(-1, inplace=True)
    hdbscan_df_test_dropna = hdbscan_df_test.copy()
    ids_test = hdbscan_df_test['id'].tolist()
except:
    pass

# if 'KL-Score'  is na, fill with 'KL-Score2'
# hdbscan_df['KL-Score'] = hdbscan_df['KL-Score'].fillna(hdbscan_df['KL-Score2'])
# hdbscan_df['KL-Score2'] = hdbscan_df['KL-Score2'].fillna(hdbscan_df['KL-Score'])

hdbscan_df['KL-Score'].fillna(-1, inplace=True)
# hdbscan_df['KL-Score2'].fillna(-1, inplace=True)
ids = hdbscan_df['id'].tolist()


hdbscan_df_dropna = hdbscan_df.copy()
# hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score'] != -1]
# hdbscan_df_dropna = hdbscan_df_dropna[hdbscan_df_dropna['KL-Score2'] != -1]

In [ ]:
with open(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)
if methods == 'comb_modalities':
    umap_info = model_info['params']['umap']
    nneigh = umap_info['n_neighbors']
    min_dist = umap_info['min_dist']
    metric = umap_info['metric']

    umap_folder = f'nneigh{nneigh}_mindist{min_dist}_metric{metric}'
    umap_path = os.path.join(proc_dir, '2026-01-16_umap_scaler_values', 'comb_modalities', umap_folder)

    embeddings_path = os.path.join(umap_path, 'X_umap_embeddings.npy')
    embeddings = np.load(embeddings_path)

    embeddings_path_test = os.path.join(filepath, 'X_test_umap_embeddings.npy')
    embeddings_test = np.load(embeddings_path_test)
else:
    embeddings_path = os.path.join(filepath, "X_umap_embeddings.npy")
    embeddings = np.load(embeddings_path)




In [ ]:
clusters = hdbscan_df['cluster_label'].unique().tolist()
clusters.sort()

healthy_cluster = 0
healthy_centroid = embeddings[hdbscan_df['cluster_label'] == healthy_cluster].mean(axis=0)
healthy_spread = np.mean(np.linalg.norm(embeddings[hdbscan_df['cluster_label'] == healthy_cluster]-healthy_centroid, axis=1))

cluster_centroids = {}

for cluster in clusters:
    cluster_embeddings = embeddings[hdbscan_df['cluster_label'] == cluster]
    cluster_centroid = cluster_embeddings.mean(axis=0)
    spread = np.mean(np.linalg.norm(cluster_embeddings-cluster_centroid, axis=1))
    cluster_centroids[cluster] = {
        'centroid': cluster_centroid,
        'distance': np.linalg.norm(cluster_centroid-healthy_centroid),
        'spread': spread,
        'norm_distance': np.linalg.norm(cluster_centroid - healthy_centroid)/(healthy_spread+spread)
    }

In [ ]:
# from scipy.spatial.distance import cdist

In [ ]:
# healthy_mask = hdbscan_df['cluster_label'] == healthy_cluster
# healthy_embeddings = embeddings[healthy_mask]
# healthy_centroid = healthy_embeddings.mean(axis=0)

# healthy_spread = np.mean(
#     np.linalg.norm(
#         healthy_embeddings - healthy_centroid,
#         axis=1
#     )
# )
# dist_to_healthy = cdist(embeddings, healthy_embeddings)

# point_severity = dist_to_healthy.min(axis=1)

# cluster_centroids = {}

# for cluster in clusters:
#     cluster_mask = hdbscan_df['cluster_label'] == cluster
#     cluster_embeddings = embeddings[cluster_mask]

#     # centroid (for reference / plotting)
#     cluster_centroid = cluster_embeddings.mean(axis=0)

#     # spread = mean distance to centroid
#     spread = np.mean(
#         np.linalg.norm(cluster_embeddings - cluster_centroid, axis=1)
#     )

#     # NEW: cluster distance = mean nearest-healthy distance
#     cluster_distance = point_severity[cluster_mask].mean()
#     print(f"Cluster {cluster}: distance to healthy = {cluster_distance}, spread = {spread}")

#     # normalized distance
#     denom = healthy_spread + spread
#     norm_distance = cluster_distance / denom if denom > 0 else np.nan

#     cluster_centroids[cluster] = {
#         'centroid': cluster_centroid,
#         'distance': cluster_distance,
#         'spread': spread,
#         'norm_distance': norm_distance
#     }


# # hdbscan_df['severity_score'] = point_severity

# # hdbscan_df['severity_score_norm'] = np.where(
# #     healthy_spread
# #     + hdbscan_df['cluster_label'].map(
# #         lambda c: cluster_centroids.get(c, {}).get('spread', np.nan)
# #     ).values
# #     > 0,
# #     point_severity
# #     / (
# #         healthy_spread
# #         + hdbscan_df['cluster_label'].map(
# #             lambda c: cluster_centroids.get(c, {}).get('spread', np.nan)
# #         ).values
# #     ),
# #     np.nan
# # )

In [ ]:
def normalized_distance(row, embeddings):
    c = row['cluster_label']
    point = embeddings[row.name]

    raw_distance = np.linalg.norm(point - healthy_centroid)
    norm = cluster_centroids[c]['spread'] + cluster_centroids[healthy_cluster]['spread']

    return raw_distance / norm if norm >0 else np.nan

In [ ]:
hdbscan_df['severity_score'] = np.linalg.norm(embeddings - healthy_centroid, axis=1)
hdbscan_df['severity_score_norm'] = hdbscan_df.apply(normalized_distance, axis = 1, embeddings= embeddings)
hdbscan_df['severity_score_norm_cluster'] = hdbscan_df['cluster_label'].apply(lambda x: cluster_centroids[x]['norm_distance'])
hdbscan_df['severity_score_manhatten'] = np.abs(np.linalg.norm(embeddings - healthy_centroid, ord=1, axis=1))
hdbscan_df['severity_score_chebyshev'] = np.abs(np.linalg.norm(embeddings - healthy_centroid, ord=np.inf, axis=1))

In [ ]:
hdbscan_df_test['severity_score'] = np.linalg.norm(embeddings_test - healthy_centroid, axis=1)
hdbscan_df_test['severity_score_norm'] = hdbscan_df_test.apply(normalized_distance, axis = 1, embeddings= embeddings_test)
hdbscan_df_test['severity_score_norm_cluster'] = hdbscan_df_test['cluster_label'].apply(lambda x: cluster_centroids[x]['norm_distance'])

In [ ]:
hdbscan_df.groupby("cluster_label")["severity_score_norm_cluster"].describe()

In [ ]:
hdbscan_df.groupby("cluster_label")["severity_score"].describe()

In [ ]:
hdbscan_df.groupby("cluster_label")["severity_score_norm"].describe()

In [ ]:
hdbscan_df.groupby("cluster_label")["severity_score_manhatten"].describe()

In [ ]:
hdbscan_df.groupby("cluster_label")["severity_score_chebyshev"].describe()

Calculate for each cluster a middle, then calculate the distance from there

In [ ]:
orderd = sorted(cluster_centroids.items(), key=lambda x: x[1]['distance'])
order_wonoised = [item for item in orderd if item[0] != -1]

In [ ]:
order = [0, 2, -1, 4, 3, 1]
order_wonoise =  [0, 2, 4, 3, 1]

# Severity Score Behaviour for each cluster label

In [ ]:
# order = [0, 4, 2, 1]
# order_wnoise = [0,4,2,-1,1]

In [ ]:
cluster_mapping = {old: new for new, old in enumerate(order_wonoise)}
print(cluster_mapping)

In [ ]:
hdbscan_df['old_cluster'] = hdbscan_df['cluster_label']
hdbscan_df['cluster_label'] = hdbscan_df['cluster_label'].map(cluster_mapping)

In [ ]:
#replace cluster_label Nan with -1
hdbscan_df['cluster_label'].fillna(-1, inplace=True)

try:
    hdbscan_df_test['old_cluster'] = hdbscan_df_test['cluster_label']
    hdbscan_df_test['cluster_label'] = hdbscan_df_test['cluster_label'].map(cluster_mapping)
    #replace cluster_label Nan with -1
    hdbscan_df_test['cluster_label'].fillna(-1, inplace=True)
except:
    pass


In [ ]:
import seaborn as sns

In [ ]:
sns.violinplot(data=hdbscan_df, x="cluster_label", y="severity_score")

In [ ]:
sns.violinplot(data=hdbscan_df_test, x="cluster_label", y="severity_score")

In [ ]:
hdbscan_df['train_test'] = 'train'
hdbscan_df_test['train_test'] = 'test'

df = pd.concat([hdbscan_df, hdbscan_df_test], ignore_index=True)

df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_severity_scores.csv'), index=False)

# SMOTE

In [ ]:
with open(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_model_info.json')) as f:
    model_info= json.load(f)
if methods == 'comb_modalities':
    umap_info = model_info['params']['umap']
    nneigh = umap_info['n_neighbors']
    min_dist = umap_info['min_dist']
    metric = umap_info['metric']

    umap_folder = f'nneigh{nneigh}_mindist{min_dist}_metric{metric}'
    umap_path = os.path.join(proc_dir, '2026-01-16_umap_scaler_values', 'comb_modalities', umap_folder)

    embeddings_path = os.path.join(umap_path, 'X_umap_embeddings.npy')
    embeddings = np.load(embeddings_path)

    embeddings_path_test = os.path.join(filepath, 'X_test_umap_embeddings.npy')
    embeddings_test = np.load(embeddings_path_test)
else:
    embeddings_path = os.path.join(filepath, "X_umap_embeddings.npy")
    embeddings = np.load(embeddings_path)




In [ ]:
import joblib

In [ ]:
smote_path = os.path.join(umap_path, 'SMOTE_generated_samples.csv')
smote_df = pd.read_csv(smote_path)
smote_embeddings = np.load(os.path.join(umap_path, 'X_umap_samp_embeddings.npy'))
#load clusterer
clusterer = joblib.load(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_scaled_clusterer.pkl'))

In [ ]:
smote_df['cluster_label']= list(clusterer.labels_)

In [ ]:
smote_df['old_label'] = smote_df['cluster_label']
smote_df['cluster_label'] = smote_df['cluster_label'].map(cluster_mapping)
smote_df['severity_score'] = np.linalg.norm(smote_embeddings - healthy_centroid, axis=1)

In [ ]:
smote_df.to_csv(os.path.join(filepath, f'{methods}_{run}_umap_hdbscan_smote_clusters.csv'), index=False)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import hdbscan
# from umap import UMAP
# import joblib

In [ ]:
# scaler_path = os.path.join(proc_dir, folder, 'pipeline', run, 'scaler.pkl')
# umapmodel_path = os.path.join(proc_dir, folder, 'pipeline', run, 'umap_model.pkl')
# hdbscan_path = os.path.join(proc_dir, folder, 'pipeline', run, 'pipeline_run10_umap_hdbscan_scaled_clusterer.pkl')

In [ ]:
# scaler = joblib.load(scaler_path)
# umap_model = joblib.load(umapmodel_path)
# clusterer = joblib.load(hdbscan_path)

In [ ]:
# X_scaled = scaler.transform(smote.drop(columns=[ 'KL-Score']))
# X_umap = umap_model.transform(X_scaled)

# cluster_labels, strengths = hdbscan.approximate_predict(clusterer, X_umap)
# smote['cluster_label'] = cluster_labels

In [ ]:
# smote['old_cluster'] = smote['cluster_label']
# smote['cluster_label_name'] = smote['cluster_label'].astype(str).map(rating)
# smote['cluster_label']=smote['cluster_label_name'].map(rating_order)

In [ ]:
# embedding_matrix = X_umap

# healthy_centroid = embedding_matrix[smote['cluster_label'] == 0].mean(axis=0)
# smote['severity_score'] = np.linalg.norm(embedding_matrix - healthy_centroid, axis=1)

In [ ]:
# summary = smote.groupby('cluster_label')['severity_score'].describe().reset_index()

In [ ]:
# summary.sort_values('mean')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=smote_df,
    x="cluster_label",
    y="severity_score",
    palette="Set3",
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)

# # add jittered points for visibility
# sns.stripplot(
#     data=smote,
#     x="cluster_label",
#     y="severity_score",
#     color="black",
#     alpha=0.4,
#     size=3,
#     jitter=0.25
# )

plt.title("Severity Score Distribution per Cluster", fontsize=16, pad=15)
plt.xlabel("Cluster Label", fontsize=13)
plt.ylabel("Severity Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(
    data=smote_df,
    x="old_label",
    y="severity_score",
    palette="Set3",
    width=0.6,
    showfliers=False,     # hide outliers
    linewidth=1.5         # thicker box borders
)

plt.title("Severity Score Distribution per Cluster", fontsize=16, pad=15)
plt.xlabel("Cluster Label", fontsize=13)
plt.ylabel("Severity Score", fontsize=13)
plt.grid(axis="y", alpha=0.25)
plt.tight_layout()
plt.show()


# Plot original data & SMOTE in embedding space

In [ ]:
import shutil


In [ ]:
output_path = os.path.join(DATAPATH, 'outputs', 'clusterimages3', f'{methods}_{run}_umap_hdbscan_cluster_label')
os.makedirs(output_path, exist_ok=True)
img_path = os.path.join(base_dir, 'images_knee')

In [ ]:
for cluster in df['cluster_label'].unique():
    ids = df[df['cluster_label'] == cluster]['id'].tolist()
    cluster_dir = os.path.join(output_path, f'cluster_{cluster}')
    os.makedirs(cluster_dir, exist_ok=True)
    for id_ in ids:
        img_file = os.path.join(img_path, f'{id_}.png')
        if os.path.exists(img_file):
            dest_file = os.path.join(cluster_dir, f'{id_}.png')
            if not os.path.exists(dest_file):
                shutil.copy2(img_file, dest_file)

In [ ]:
output_path2 = os.path.join(DATAPATH, 'outputs', 'clusterimages3', f'{methods}_{run}_umap_hdbscan_old_label')
os.makedirs(output_path2, exist_ok=True)
img_path = os.path.join(base_dir, 'images_knee')

In [ ]:
for cluster in df['old_cluster'].unique():
    ids = df[df['old_cluster'] == cluster]['id'].tolist()
    cluster_dir = os.path.join(output_path2, f'cluster_{cluster}')
    os.makedirs(cluster_dir, exist_ok=True)
    for id_ in ids:
        img_file = os.path.join(img_path, f'{id_}.png')
        if os.path.exists(img_file):
            dest_file = os.path.join(cluster_dir, f'{id_}.png')
            if not os.path.exists(dest_file):
                shutil.copy2(img_file, dest_file)

In [ ]:
# hdbscan_doc = pd.read_csv(os.path.join(filepath, f'pipeline_run10_umap_hdbscan_scaled_allpoints_wKL.csv'))

In [ ]:
# hdbscan_doc['clusterlabelold'] = hdbscan_doc['cluster_label']
# hdbscan_doc['clusterlabeltext'] = hdbscan_doc['cluster_label'].astype(str).map(rating)
# hdbscan_doc['cluster_label'] = hdbscan_doc['clusterlabeltext'].map(rating_order)

# Get images per cluster

In [ ]:
import random
import matplotlib.pyplot as plt
from PIL import Image

def get_image_id(img_path):
    return os.path.splitext(os.path.basename(img_path))[0]


def show_cluster_examples(output_path, df, max_grid=3, seed=42):
    random.seed(seed)

    # Build KL lookup
    kl_lookup = dict(zip(df['name'], df['KL-Score']))

    clusters = sorted([c for c in os.listdir(output_path) if c.startswith("cluster_")])
    img_names_l = []

    for cluster in clusters:
        cluster_dir = os.path.join(output_path, cluster)

        imgs = [
            os.path.join(cluster_dir, f)
            for f in os.listdir(cluster_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]

        if len(imgs) == 0:
            print(f"No images found in {cluster}")
            continue

        # -----------------------------
        # Group images by KL-score
        # -----------------------------
        imgs_by_kl = {}
        for img_path in imgs:
            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, None)
            if kl is None:
                continue
            imgs_by_kl.setdefault(kl, []).append(img_path)

        available_kls = sorted(imgs_by_kl.keys())

        if len(available_kls) == 0:
            print(f"No KL-labelled images in {cluster}")
            continue

        # -----------------------------
        # Ensure at least one per KL
        # -----------------------------
        sample_imgs = []

        for kl in available_kls:
            sample_imgs.append(random.choice(imgs_by_kl[kl]))

        # -----------------------------
        # Fill remaining slots (if any)
        # -----------------------------
        max_images = max_grid * max_grid
        remaining = max_images - len(sample_imgs)

        if remaining > 0:
            remaining_imgs = [
                img for img in imgs
                if img not in sample_imgs
            ]

            if len(remaining_imgs) > 0:
                sample_imgs.extend(
                    random.sample(
                        remaining_imgs,
                        min(remaining, len(remaining_imgs))
                    )
                )

        img_names_l.append(sample_imgs)

        # -----------------------------
        # Plot
        # -----------------------------
        n_images = len(sample_imgs)
        cols = min(max_grid, n_images)
        rows = (n_images + cols - 1) // cols

        fig, axes = plt.subplots(rows, cols, figsize=(4 * cols, 4 * rows))
        fig.suptitle(
            f"Cluster {cluster.replace('cluster_', '')}: Example Images",
            fontsize=18
        )

        if rows == 1 and cols == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes
        else:
            axes = axes.flatten()

        for ax, img_path in zip(axes, sample_imgs):
            img = Image.open(img_path).convert("L")
            ax.imshow(img, cmap="gray")
            ax.axis("off")

            img_id = get_image_id(img_path)
            kl = kl_lookup.get(img_id, "KL ?")

            ax.text(
                0.02, 0.02,
                f"KL {kl}",
                transform=ax.transAxes,
                fontsize=14,
                color="white",
                ha="left",
                va="bottom",
                bbox=dict(facecolor="black", alpha=0.5, pad=2)
            )

        # Hide unused axes
        for ax in axes[len(sample_imgs):]:
            ax.axis("off")

        plt.tight_layout()
        plt.show()

    #return img_names_l


In [ ]:
show_cluster_examples(output_path, df, max_grid=3, seed=8)

In [ ]:
show_cluster_examples(output_path2, df, max_grid=3, seed=8)